In [ ]:
!pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118
!pip install numpy==1.26.4
!pip install ultralytics

In [ ]:
# ───────────────────────────────
# 0. 掛載與資料準備
# ───────────────────────────────
import os, shutil, yaml
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# 設定路徑
base_drive = '/content/drive/MyDrive/Competition2/Competition/'
local_path = '/content/dataset_column/column_crack/'

# 複製訓練資料
shutil.copytree(base_drive + 'dataset_column/column_crack/', local_path, dirs_exist_ok=True)


In [ ]:
import os
from collections import Counter

# label.txt 的 class_id 對應到實際 criteria 原始 ID 的順序（根據你的 names）
class_id_to_original_id = {
    0: 0,  # rebar
    1: 3,  # xv
    2: 4,  # cdiag
    3: 5,  # ddiag
    4: 6,  # cvert
    5: 7,  # dvert
    6: 8   # chori
}

# 類別名稱列表（對應 class_id）
class_names = [
    '0_rebar',  # 0 → 原 ID 0
    '3_xv',     # 1 → 原 ID 3
    '4_cdiag',  # 2 → 原 ID 4
    '5_ddiag',  # 3 → 原 ID 5
    '6_cvert',  # 4 → 原 ID 6
    '7_dvert',  # 5 → 原 ID 7
    '8_chori'   # 6 → 原 ID 8
]

# label 路徑（你訓練集的標註資料夾）
label_dir = '/content/dataset_column/column_crack/train/labels'

# 計數器初始化
criteria_counter = Counter()

# 統計每個 class_id 出現次數
for file in os.listdir(label_dir):
    if file.endswith('.txt'):
        with open(os.path.join(label_dir, file)) as f:
            for line in f:
                parts = line.strip().split()
                if parts and parts[0].isdigit():
                    cid = int(parts[0])
                    if 0 <= cid <= 8:
                        criteria_counter[cid] += 1

# 顯示結果（依照 class_id 顯示對應的原始 ID 與名稱）
print("📊 各類別（criteria）出現次數：")
for cid in sorted(criteria_counter.keys()):
    real_id = class_id_to_original_id[cid]
    name = class_names[cid]
    count = criteria_counter[cid]
    print(f"  class_id={cid:>2} → 原ID={real_id:>2} → {name:<10}：{count} 次")


In [ ]:
# ───────────────────────────────
# 1. 建立 YOLOv8 訓練資料 data.yaml
# ───────────────────────────────
data_yaml = {
    # 'path': '/content/drive/MyDrive/Competition2/Competition/dataset_column/column_crack',
    'path': local_path,
    'train': 'train/images',
    'val': 'train/images',
    'nc': 7,
    'names': [
        '0_rebar',   # 0 → 0
        '3_xv',      # 1 → 3
        '4_cdiag',   # 2 → 4
        '5_ddiag',   # 3 → 5
        '6_cvert',   # 4 → 6
        '7_dvert',   # 5 → 7
        '8_chori'    # 6 → 8
    ]
}

with open('column_data_yolov8.yaml', 'w') as f:
    yaml.dump(data_yaml, f)


In [ ]:
# ───────────────────────────────
# 2. 訓練 YOLOv8 模型
# ───────────────────────────────
from ultralytics import YOLO

model = YOLO('yolov8m.pt')
model.train(
    data='column_data_yolov8.yaml',
    imgsz=640,
    epochs=200,
    batch=8,
    name='yolov8_column_crack_aug',
    project='runs_yolov8',
    device=0,
    cos_lr=True,
    amp=True,
    save=True,
    save_period=10,
    warmup_epochs=3,
    weight_decay=0.0005,

    # 自定義 資料增強
    hsv_h=0.05,
    hsv_s=0.8,
    hsv_v=0.5,
    degrees=10,
    translate=0.2,
    scale=0.8,
    shear=5,
    flipud=0.2,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.3,
    copy_paste=0.2,
)


In [ ]:
# ───────────────────────────────
# 3. 進行測試資料推論，儲存 txt 預測結果
# ───────────────────────────────
from ultralytics import YOLO

# 請將 best.pt 路徑換成實際訓練結果的路徑
model = YOLO('/content/runs_yolov8/yolov8_column_crack_aug/weights/best.pt')

_ = model.predict(
    source=base_drive + '/dataset_column/test',
    save=True,
    save_txt=True,
    save_conf=True,
    conf=0.001,
    imgsz=640,
    project='runs/detect',
    name='predict'
)


In [ ]:
from ultralytics import YOLO
import os, cv2
import matplotlib.pyplot as plt
import pandas as pd

cmap = {
    'w': '/content/drive/MyDrive/Competition2/best.pt',
    'f': '/content/drive/MyDrive/Competition2/Competition/dataset_column/test',
    'o': '/content/column_submission.csv'
}
sieve = YOLO(cmap['w'])

nametag = ['0_rebar', '3_xv', '4_cdiag', '5_ddiag', '6_cvert', '7_dvert', '8_chori']
conv_map = {i: v for i, v in enumerate([0, 3, 4, 5, 6, 7, 8])}
gate = {0: .20, 1: .02, 2: .01, 3: .50, 4: .01, 5: .20, 6: .05}
decision = {18: {0, 3, 4, 6, 8}, 19: {5, 7}, 20: {1, 9, 10}}
container = []

for k in range(1, 51):
    p = os.path.join(cmap['f'], f"{k}.jpg")
    frame, memo = [], []
    if not os.path.exists(p):
        print(f"🟥 Skip {p}")
        container.append([k, '20,1'])
        continue

    probe = sieve(p, conf=0.001)[0]
    scr = cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2RGB)

    for b in probe.boxes:
        cid, cof = int(b.cls[0]), float(b.conf[0])
        if cof >= gate.get(cid, 1):
            rid = conv_map[cid]
            frame.append(rid)
            memo.append((rid, cof))
            x1, y1, x2, y2 = map(int, b.xyxy[0])
            cv2.rectangle(scr, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(scr, f"{nametag[cid]} {cof:.2f}", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    if 0 in frame:
        z, t = 18, [0]
    elif memo:
        m = sorted(memo, key=lambda z: -z[1])[0][0]
        z = next((c for c, pool in decision.items() if m in pool), 20)
        valid = decision[z]
        t = sorted(set(i for i, s in memo if i in valid))
    else:
        z, t = 18, [3,4]

    plt.figure(figsize=(8, 5))
    plt.imshow(scr)
    plt.axis('off')
    plt.title(f"{k}.jpg → class={z}, criteria={t}")
    plt.show()

    s = ','.join(map(str, t))
    container.append([k, f"{z},{s}"])

# === 結果輸出 ===
pd.DataFrame(container, columns=["ID", "class"]).to_csv(cmap['o'], index=False)
print(f"✅ 結果已儲存於 {cmap['o']}")
